In [2]:
pip install SPARQLWrapper

  Obtaining dependency information for SPARQLWrapper from https://files.pythonhosted.org/packages/31/89/176e3db96e31e795d7dfd91dd67749d3d1f0316bb30c6931a6140e1a0477/SPARQLWrapper-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for rdflib>=6.1.1 from https://files.pythonhosted.org/packages/9c/3c/f508a9b346078ea0bd49c8261430204fcfb4150352d51fa2a54a4d9eacda/rdflib-7.1.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 4.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

In [13]:
# Specify your input and output file paths
input_file_path = '/Users/mel/Downloads/input.csv'
output_file_path = '/Users/mel/Downloads/output.csv'

In [28]:
df = pd.read_csv(input_file_path)
df.head()

,adminEntity_qcode
0,Q1000
1,Q1000
2,Q1006
3,Q1008
4,Q1019


In [29]:
len(df)

577

In [17]:
def get_p131_info(qcode):
    """
    Given a Wikidata Q-code (e.g. 'Q1042'),
    returns (p131_url, p131_qcode) if found,
    otherwise (None, None).
    """
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    
    query = f"""
    SELECT ?p131
    WHERE {{
      wd:{qcode} wdt:P131 ?p131 .
    }}
    LIMIT 1
    """
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        results = sparql.query().convert()
        bindings = results["results"]["bindings"]
        
        if bindings:
            p131_url = bindings[0]["p131"]["value"]  # e.g. "http://www.wikidata.org/entity/QXXXX"
            p131_qcode = p131_url.split("/")[-1]     # just the "QXXXX" part
            return p131_url, p131_qcode
        else:
            return None, None
        
    except Exception as e:
        print(f"Error querying P131 for {qcode}: {e}")
        return None, None

In [30]:
df["lookup_wikidata_url"] = None
df["lookup_wikidata_qcode"] = None

In [31]:
for idx, row in df.iterrows():
    qcode = row["adminEntity_qcode"]
    p131_url, p131_qcode = get_p131_info(qcode)
    
    df.at[idx, "lookup_wikidata_url"] = p131_url
    df.at[idx, "lookup_wikidata_qcode"] = p131_qcode

In [32]:
df.head()

,adminEntity_qcode,lookup_wikidata_url,lookup_wikidata_qcode
0,Q1000,None,None
1,Q1000,None,None
2,Q1006,None,None
3,Q1008,None,None
4,Q1019,None,None


In [33]:
# If you want to write it back to a CSV:
df.to_csv(output_file_path, index=False)